In [1]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import ipywidgets as widgets
from IPython.display import display
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact ## para análisis interactivo
from sklearn import neighbors ### basado en contenido un solo producto consumido
import joblib
#### conectar_base_de_Datos

In [2]:

conn=sql.connect('db_movies') ### crear cuando no existe el nombre de cd  y para conectarse cuando sí existe.
cur=conn.cursor() ###para funciones que ejecutan sql en base de datos


RECOMENDACIONES BASADAS EN POPULARIDAD


In [3]:
#### Peliculas mejor calificados por año de publicacion con más de 50 calificaciones ###
pd.read_sql("""select año_estreno, title, 
            avg(iif(movie_rating = 0, Null, movie_rating)) as avg_rat,
            count(iif(movie_rating = 0, Null, movie_rating)) as conteo_rat
            from ratings_finalready
            group by  año_estreno, title
            having conteo_rat >50
            order by año_estreno desc, avg_rat desc limit 30
            """, conn)


,año_estreno,title,avg_rat,conteo_rat
0,2016,Deadpool (2016),3.833333,54
1,2014,Guardians of the Galaxy (2014),4.050847,59
2,2014,Interstellar (2014),3.993151,73
3,2014,"Grand Budapest Hotel, The (2014)",3.778846,52
4,2013,"Wolf of Wall Street, The (2013)",3.916667,54
5,2012,"Dark Knight Rises, The (2012)",3.993421,76
6,2012,Django Unchained (2012),3.943662,71
7,2012,"Avengers, The (2012)",3.869565,69
8,2012,The Hunger Games (2012),3.435185,54
9,2010,Toy Story 3 (2010),4.109091,55


In [4]:
### Joyas ocultas. peliculas menos calificadas con mejor puntuación ###
pd.read_sql("""
select title, 
       avg(movie_rating) as avg_rat,
       count(*) as conteo_calificaciones
from ratings_finalready
group by title
having conteo_calificaciones BETWEEN 10 AND 50
order by avg_rat desc
limit 10
""", conn)

,title,avg_rat,conteo_calificaciones
0,Secrets & Lies (1996),4.590909,11
1,Guess Who's Coming to Dinner (1967),4.545455,11
2,Paths of Glory (1957),4.541667,12
3,"Streetcar Named Desire, A (1951)",4.475000,20
4,"Celebration, The (Festen) (1998)",4.458333,12
5,Ran (1985),4.433333,15
6,His Girl Friday (1940),4.392857,14
7,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),4.333333,27
8,"Hustler, The (1961)",4.333333,18
9,Double Indemnity (1944),4.323529,17


In [5]:
### Películas con calificaciones altas de usuarios frecuentes ###
pd.read_sql("""
select title, 
       avg(movie_rating) as avg_rat,
       count(*) as conteo_calificaciones
from ratings_finalready r
join (select user_id, count(*) as conteo_peliculas
      from ratings_finalready
      group by user_id
      having conteo_peliculas > 50) usuarios_frecuentes
on r.user_id = usuarios_frecuentes.user_id
group by title
having conteo_calificaciones > 10
order by avg_rat desc
limit 10
""", conn)


,title,avg_rat,conteo_calificaciones
0,Paths of Glory (1957),4.545455,11
1,Guess Who's Coming to Dinner (1967),4.545455,11
2,"Celebration, The (Festen) (1998)",4.458333,12
3,"Shawshank Redemption, The (1994)",4.438914,221
4,"Streetcar Named Desire, A (1951)",4.437500,16
5,Ran (1985),4.433333,15
6,Patton (1970),4.428571,28
7,Wallace & Gromit: The Best of Aardman Animatio...,4.416667,24
8,His Girl Friday (1940),4.392857,14
9,"Philadelphia Story, The (1940)",4.333333,24


Sistema de recomendación basado en contenido

In [6]:
movies=pd.read_sql('select * from moviesready',conn)
movies.info()
movies['año_estreno']=movies.año_estreno.astype('int')
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2121 entries, 0 to 2120
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   movieId      2121 non-null   int64 
 1   title        2121 non-null   object
 2   cnt_rat      2121 non-null   int64 
 3   Action       2121 non-null   int64 
 4   Adventure    2121 non-null   int64 
 5   Animation    2121 non-null   int64 
 6   Children     2121 non-null   int64 
 7   Comedy       2121 non-null   int64 
 8   Crime        2121 non-null   int64 
 9   Documentary  2121 non-null   int64 
 10  Drama        2121 non-null   int64 
 11  Fantasy      2121 non-null   int64 
 12  Film-Noir    2121 non-null   int64 
 13  Horror       2121 non-null   int64 
 14  IMAX         2121 non-null   int64 
 15  Musical      2121 non-null   int64 
 16  Mystery      2121 non-null   int64 
 17  Romance      2121 non-null   int64 
 18  Sci-Fi       2121 non-null   int64 
 19  Thriller     2121 non-null 

In [7]:
##### escalador
sc=MinMaxScaler()
movies[["año_estreno_sc"]]=sc.fit_transform(movies[['año_estreno']])

##### eliminar filas que 
movies_dum1=movies.drop(columns=['movieId','title','cnt_rat','año_estreno'])

### para proximos modelos

joblib.dump(movies_dum1,"salidas_movies_dum1.joblib")


['salidas_movies_dum1.joblib']

In [8]:
##### ### entrenar modelo #####

model = neighbors.NearestNeighbors(n_neighbors=10, metric='cosine')#arroja 10 resultados
model.fit(movies_dum1)
dist, idlist = model.kneighbors(movies_dum1)


distancias=pd.DataFrame(dist) ## devuelve un ranking de la distancias más cercanas 
id_list=pd.DataFrame(idlist) ## relaciona la distancia con el item 


In [9]:
####ejemplo
movie_list_name = []
movie_name='The Lego Movie (2014)'
movie_id = movies[movies['title'] == movie_name].index ### extraer el indice
movie_id = movie_id[0] ## si encuentra varios solo guarde uno


### agrega el nombre de cada una de los id recomendados
for newid in idlist[movie_id]:
        movie_list_name.append(movies.loc[newid].title) 

movie_list_name


['The Lego Movie (2014)',
 'Zootopia (2016)',
 'Bolt (2008)',
 'Shrek the Third (2007)',
 'Incredibles, The (2004)',
 'Monsters, Inc. (2001)',
 "Emperor's New Groove, The (2000)",
 'Toy Story 2 (1999)',
 'Antz (1998)',
 'Toy Story (1995)']

In [13]:
### lista de películas recomendadas basadas en el título y filtra las recomendaciones por género ###

def recomiendamovies(movie_name=list(movies['title'].value_counts().index), genre_filter=None):
    recommended_movies = []
    
    # Obtener el índice de la película seleccionada
    movie_index = movies[movies['title'] == movie_name].iloc[0].name
    
    # Añadir las recomendaciones basadas en el idlist
    for recommended_id in idlist[movie_index]:
        movie_row = movies.loc[recommended_id]
        
        # Filtrar por género
        if genre_filter and movie_row[genre_filter] == 0:
            continue
        
        recommended_movies.append(movie_row.title)
    
    # Eliminar la película seleccionada de la lista de recomendaciones
    recommended_movies = [movie for movie in recommended_movies if movie != movie_name]
    
    # Crear un cuadro de texto con las recomendaciones
    textarea = widgets.Textarea(
        value="\n".join(recommended_movies),
        description='Recomendaciones:',
        disabled=True,  # Solo para mostrar, no editable
    )
    display(textarea)

# Uso interactivo con filtro por género
widgets.interact(recomiendamovies, 
                 genre_filter=widgets.Dropdown(options=movies.columns[3:22]))  # Filtrar solo por géneros


interactive(children=(Dropdown(description='movie_name', options=('Toy Story (1995)', 'Road to Perdition (2002…

<function __main__.recomiendamovies(movie_name=['Toy Story (1995)', 'Road to Perdition (2002)', 'Strange Brew (1983)', 'Red Dragon (2002)', 'Sweet Home Alabama (2002)', 'Spirited Away (Sen to Chihiro no kamikakushi) (2001)', 'Secretary (2002)', 'Das Experiment (Experiment, The) (2001)', 'Igby Goes Down (2002)', 'Transporter, The (2002)', 'Hot Shots! (1991)', 'One Hour Photo (2002)', 'xXx (2002)', 'Good Girl, The (2002)', 'Signs (2002)', 'Austin Powers in Goldmember (2002)', 'Reign of Fire (2002)', 'Old School (2003)', 'Men in Black II (a.k.a. MIIB) (a.k.a. MIB 2) (2002)', 'Mr. Deeds (2002)', 'Minority Report (2002)', 'Lilo & Stitch (2002)', 'Scooby-Doo (2002)', 'Bourne Identity, The (2002)', 'Sum of All Fears, The (2002)', 'Insomnia (2002)', 'Star Wars: Episode II - Attack of the Clones (2002)', 'About a Boy (2002)', 'Spider-Man (2002)', 'The Scorpion King (2002)', 'Three Men and a Baby (1987)', 'My Big Fat Greek Wedding (2002)', 'Bowling for Columbine (2002)', 'Punch-Drunk Love (2002)